# **生成数据集**

In [1]:
import numpy as np
import torch

In [2]:
nums_input = 2
nums_example = 1000
true_w = [2, -3.4]
true_b = 4.2
# 生成tensor类型的数据
features = torch.tensor(np.random.normal(0, 1, (nums_example, nums_input)), dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b + torch.tensor(np.random.normal(0, 0.01, 1000), dtype=torch.float32)

# **使用TorchAPI读取数据**

In [3]:
import torch.utils.data as Data

In [4]:
batch_size = 10
# 组合data和label
dataset = Data.TensorDataset(features, labels)
# 随机读取batch
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True) # 这是个生成器

In [5]:
for x, y in data_iter:
    print(x, y)
    break


tensor([[ 0.5238, -0.7384],
        [ 0.0806,  0.6018],
        [-0.4914, -0.3885],
        [-0.9155,  1.8756],
        [-1.4436, -0.3935],
        [-0.3330, -0.3985],
        [-1.2804,  1.0098],
        [-0.8236,  1.3284],
        [-0.2955, -0.4201],
        [ 0.6257,  0.7104]]) tensor([ 7.7444,  2.3108,  4.5283, -3.9925,  2.6395,  4.9068, -1.7886, -1.9611,
         5.0346,  3.0234])


# **定义模型**

torch.nn模块实际上包含了很多已经定义好的神经网络层     
nn的核心类叫做Module，这是个抽象概念，既可以表示单个层，也可以表示多个层连接在一起      
常见的作法是继承Module类，撰写自己的网络层，主要修改前向函数    

In [6]:
class LinearNet(torch.nn.Module):
    
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = torch.nn.Linear(n_feature, 1)
        
    def forward(self, x):
        y = self.linear(x)
        return(y)

    
net = LinearNet(nums_input)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


和kears类似，pytorch支持使用torch.nn.Sequential()来组件式搭建模型,这相当于一个有序容器
```python
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))
```

In [7]:
from collections import OrderedDict
net = torch.nn.Sequential(OrderedDict([
          ('linear', torch.nn.Linear(nums_input, 1))
          # ......
        ]))

In [8]:
# 查看可训练参数
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.4678,  0.0412]], requires_grad=True)
Parameter containing:
tensor([0.2128], requires_grad=True)


In [9]:
print(net[0])

Linear(in_features=2, out_features=1, bias=True)


# **初始化模型参数**

In [10]:
from torch.nn import init

init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0.0)

Parameter containing:
tensor([0.], requires_grad=True)

# **损失函数**

In [11]:
loss = torch.nn.MSELoss()

# **优化方法**

In [12]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.01)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)


optimizer支持使用list嵌套dict为不同层指定不同学习率

# **训练模型**

In [13]:
num_epochs = 9
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零
        l.backward()
        optimizer.step() # 一次迭代
    print('epoch{:d}:loss{:f}'.format(epoch+1, l.item()))

epoch2:loss41.057049
epoch3:loss4.761404
epoch4:loss6.574754
epoch5:loss1.751640
epoch6:loss0.605364
epoch7:loss0.221531
epoch8:loss0.083104
epoch9:loss0.031583
epoch10:loss0.010608


In [14]:
dense = net[0]

In [15]:
dense.weight

Parameter containing:
tensor([[ 1.9934, -3.3670]], requires_grad=True)

In [16]:
dense.bias

Parameter containing:
tensor([4.1056], requires_grad=True)